# Part I: the basics

Reference: https://www.dataquest.io/blog/sql-basics/

#### In this tutorial we'll be working with a dataset from the bike-sharing service Hubway. It includes data for millions of trips made with the service. 

Download the file here https://dataquest.io/blog/large_files/hubway.db

We will be using a database system called SQLite3 (part of Python, for more details see https://www.pythoncentral.io/introduction-to-sqlite-in-python/). Using Python to run our SQL code allows us to import the results into a Pandas dataframe to make it easier to display our results in an easy to read format. 

#### Let's first define a function that takes our query (stored as a string) as an input and shows the result as a formatted dataframe:

In [ ]:
import sqlite3
import pandas as pd

#connect to the database and open file
db = sqlite3.connect('hubway.db')

def run_query(query):
    #Read SQL query into a DataFrame
    return pd.read_sql_query(query,db)

### The SELECT command

Select is the most basic and frequently used command. It tells the database which columns you want to see. Let's check out some examples:

In [ ]:
#let's see the tables the database has
#and how they are called

query = "SELECT name FROM sqlite_master \
 where type='table';" #selects "name" column
run_query(query)

In [ ]:
# "*" returns every column 
query = "SELECT * FROM sqlite_master where type='table';" 
#selects "name" column
run_query(query)

The database has two tables, TRIPS and STATIONS. We will first work with the TRIPS table, which contains information (columns) on id, trip duration, start/end date, etc.

In [ ]:
query = 'SELECT * FROM trips LIMIT 5;' 
run_query(query)

In [ ]:
query = 'SELECT duration, start_date, gender FROM trips LIMIT 5;' 
run_query(query)

### The ORDER BY command

This command allows you to sort the database on a given column - default is ascending order. Let's use it to find out how long the longest trip lasted.

In [ ]:
query = '''
SELECT duration 
FROM trips
ORDER BY duration DESC
LIMIT 10;
'''

run_query(query)

So the longest trip lasts 9999 seconds (bit less than 3 hours). 

### The WHERE command

The WHERE command is used to specify a certain subset of data. For example you could use the following command to return every trip with a duration longer than 9990 seconds:

In [ ]:
query = '''
SELECT * 
FROM trips
WHERE duration > 9990;
'''

run_query(query)

#### Comment:
As you can see, the query returned 14 different trips, each with a duration of 9990 seconds or more. Something that stands out about this query is that all but one of the results has a sub_type of "Casual". Perhaps this is an indication that "Registered" users are more aware of the extra fees for long trips and maybe Hubway could do a better job of conveying their pricing structure to Casual users to help them avoid overage charges.

#### Let's use AND to specify two conditions: duration > 9990 and Registered user:

In [ ]:
query = '''
SELECT * 
FROM trips
WHERE (duration >= 9990) AND (sub_type = "Registered")
'''

run_query(query)

#### Now let's answer the question "How many trips were taken by registered users". We will use the COUNT command:

In [ ]:
query = '''
SELECT COUNT(id)
FROM trips
WHERE sub_type = "Registered";
'''

run_query(query)

#### Use AS to make this more informative/readable:

In [ ]:
query = '''
SELECT COUNT(id) AS "Total Trips by Registered Users"
FROM trips
WHERE sub_type = "Registered";
'''

run_query(query)

### Aggregate Functions

Aggregate functions include COUNT, SUM (returns the sum), AVG (returns the average), MIN (returns the minimum), MAX (returns the maximum). 

In [ ]:
query = '''
SELECT AVG(duration) AS "Average Duration"
FROM trips;
'''

run_query(query)

In [ ]:
query = '''
SELECT MIN(duration) AS "Minimum Duration"
FROM trips;
'''

run_query(query)

### The GROUP BY command

GROUP BY separates the rows into groups based on the contents of a particular column and allows us to perform aggregate functions on each group. We'll use this to write a query to answer the question of whether registered or casual users take longer trips.

In [ ]:
query = '''
SELECT sub_type, AVG(duration) AS "Average Duration"
FROM trips
GROUP BY sub_type; 
'''
# GROUP BY sub_type means the averages of 
# registered and casual users are calculated separately

run_query(query)

#### Now let's answer the question of which bike was used for the most trips:

In [ ]:
query = '''
SELECT bike_number as "Bike Number", COUNT(*) 
AS "Number of Trips"
FROM trips
GROUP BY bike_number
ORDER BY COUNT(*) DESC
LIMIT 1;
'''

run_query(query)

### Arithmetic Operators

SQL allows us to use arithmetic operators.
Let's use them to calculate the average duration of trips by registered members under the age of 40:

In [ ]:
query = '''
SELECT AVG(duration)
FROM trips
WHERE (2018 - birth_date) < 40;
'''

run_query(query)

So far we've been looking at queries that only pull data from the trips table, but you'll remember from the introduction that this database contains a second table, stations. The stations table contains information about every station in the Hubway network and includes an id column that is referenced by the trips table.

#### Let's have a look:

In [ ]:
query = '''
SELECT *
FROM stations
LIMIT 3;
'''
run_query(query)

#### ID is a unique identifier for each station (corresponds to the start_station and end_station columns in the trips table).

#### We want to know which station is the most frequent starting point. For that we will use the JOIN command.

### JOIN

JOIN helps us query information that is stored in different tables.

First we want to use SELECT to return the station column from the stations table and the COUNT of the number of rows. To specify which column is in which table we use 

#### table.column 

So we need to use

#### stations.station

To tell the database how the stations and trips tables are connected, we use JOIN and ON.
JOIN specifies which tables should be connected and ON specifies which columns in each table are related. We're going to use an INNER JOIN, which means that rows will only be returned where there is a match in our columns specified in ON.

So we tell the database to connect the tables ON the start_station column in the trips table and the id column in the stations table.

Then we GROUP BY the station column in the stations table so that our COUNT will count up the number of trips for each station separately, and we ORDER BY descending order.

In [ ]:
query = '''
SELECT stations.station AS "Station", COUNT(*) AS "Count"
FROM trips 
INNER JOIN stations
ON trips.start_station = stations.id
GROUP BY stations.station
ORDER BY COUNT(*) DESC
LIMIT 5;
'''

run_query(query)

#### Let's slightly extend this query to see which stations are most frequently used for round trips:

In [ ]:
query = '''
SELECT stations.station AS "Station", COUNT(*) AS "Count"
FROM trips 
INNER JOIN stations
ON trips.start_station = stations.id
WHERE trips.start_station = trips.end_station
GROUP BY stations.station
ORDER BY COUNT(*) DESC
LIMIT 5;
'''

run_query(query)

## Tutorial Exercises

#### [1] Display the TRIPS information for the duration, subscription type, and gender.

#### [2] Some gender values are null. Display the previous table with the null gender rows omitted. First 10 rows only.

#### [3] Same as before, but only the lines who have Female gender.

#### [4] How many trips lasted more than half an hour, which means they incurred additional fees? Write two queries: One for seeing all the information about these trips, and one showing only their number.

Half an hour = 30 minutes = 1800 seconds

#### [5] Same as before, but also order by descending duration of trip.

#### [6] Which station do the shortest trips start from? Note we want the station's name, not just the number.

#### [7] How many trips start and end in different municipalities? 

#### [8] Create a subquery to only show the duration and end_station information from start station 144.

#### [9] Combine start and end station numbers in one column separated by a hyphen. Show this column together with id and duration only.

#### DSI SQL tutorial, 2018